In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from matplotlib.ticker import ScalarFormatter
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
import os

if os.name == 'posix':
    plt.rc("font", family="AppleGothic")
else:
    plt.rc("font", family="Malgun Gothic")

In [3]:
df = pd.read_csv('C:/Users/Home/Desktop/Data/2차/CardData_new.csv', encoding = 'cp949')

In [5]:
d = {
    '21.Active_연속' : 0,
    '22.Active_간헐' : 1,
    '34.휴면_9M+' : 2,
    '32.휴면_6M-' : 2,
    '33.휴면_9M-' : 2,
    '31.휴면_3M' : 2
}
df['고객구분'] = df['고객구분'].map(d)

In [6]:
df2 = df.copy()

In [7]:
df2['이용건수_신용_B0M'].value_counts()

0      62116
1      38308
2      15646
3      10565
4       8461
       ...  
245        1
318        1
355        1
261        1
260        1
Name: 이용건수_신용_B0M, Length: 271, dtype: int64

In [15]:
df2['이용건수_체크_B0M'].value_counts()

 0      259808
 1        5588
 2        3168
 3        2187
 4        1744
         ...  
 282         1
 168         1
 158         1
 302         1
-28          1
Name: 이용건수_체크_B0M, Length: 191, dtype: int64

In [136]:
df2['Recency'] = (df2['이용건수_신용_B0M'].apply(lambda x: 0 if x <= 0 else 1) +
                 df2['이용건수_체크_B0M'].apply(lambda x: 0 if x <= 0 else 1) +
                 df2['이용건수_카드론_B0M'].apply(lambda x: 0 if x <= 0 else 1) +
                 df2['이용건수_카드론_R3M'].apply(lambda x: 0 if x <= 0 else 1) +
                 df2['RP건수_B0M'].apply(lambda x: 0 if x <= 0 else 1) +
                 df2['이용건수_온라인_B0M'].apply(lambda x: 0 if x <= 0 else 1) +
                 df2['이용건수_오프라인_B0M'].apply(lambda x: 0 if x <= 0 else 1) +
                 df2['이용건수_페이_오프라인_R3M'].apply(lambda x: 0 if x <= 0 else 1) +
                 df2['이용건수_페이_온라인_B0M'].apply(lambda x: 0 if x <= 0 else 1) +
                 df2['이용건수_페이_오프라인_B0M'].apply(lambda x: 0 if x <= 0 else 1))

In [70]:
df2['Recency'].value_counts().sort_index()

0     54091
1      7604
2     43771
3     87611
4     37835
5     41597
6     13884
7     11222
8      2250
9       113
10       22
Name: Recency, dtype: int64

In [137]:
df2['Frequency'] = df2['이용건수_신용_R12M'] + df2['이용건수_체크_R12M'] + df2['이용건수_카드론_R12M']

In [138]:
df2['Monetary'] = df2['이용금액_신용_R12M'] + df2['이용금액_체크_R12M'] + df2['이용금액_카드론_R12M'] + df2['이용금액_업종기준']

In [73]:
df2['이용후경과월_신용'].value_counts().sort_index()

0     237508
1       8844
2       5546
3       4539
4       3309
5       2951
6       2691
7       2394
8       2038
9       1832
10      1790
11      1626
12     24932
Name: 이용후경과월_신용, dtype: int64

In [90]:
df2['Recency'].value_counts().sort_index()

0     54091
1      7604
2     43771
3     87611
4     37835
5     41597
6     13884
7     11222
8      2250
9       113
10       22
Name: Recency, dtype: int64

In [139]:
df2['Recency_5'] = df2['Recency'].map(lambda x : {0 : 5, 1 : 5, 2 :4, 3 : 4, 4: 3, 5:3, 6:2, 7:2, 8:1, 9:1, 10:1}[x])

In [106]:
df2['Frequency'].quantile(0.8)

434.0

In [140]:
df2['Frequency_5'] = pd.cut(df2['Frequency'], bins = [df2['Frequency'].quantile(0), df2['Frequency'].quantile(0.2), df2['Frequency'].quantile(0.4),
                                 df2['Frequency'].quantile(0.6), df2['Frequency'].quantile(0.8), df2['Frequency'].quantile(1)],
      labels = [5, 4, 3, 2, 1], include_lowest = True)

In [127]:
pd.cut(df2['Frequency'], bins = [df2['Frequency'].quantile(0), df2['Frequency'].quantile(0.2), df2['Frequency'].quantile(0.4),
                                 df2['Frequency'].quantile(0.6), df2['Frequency'].quantile(0.8), df2['Frequency'].quantile(1)],
      labels = [5, 4, 3, 2, 1], include_lowest = True).value_counts()

5    61253
1    59997
2    59906
3    59529
4    59315
Name: Frequency, dtype: int64

In [96]:
df2['Recency'].quantile(0.8)

5.0

In [141]:
df2['Monetary_5'] = pd.cut(df2['Monetary'], bins = [df2['Monetary'].quantile(0), df2['Monetary'].quantile(0.2), df2['Monetary'].quantile(0.4),
                                 df2['Monetary'].quantile(0.6), df2['Monetary'].quantile(0.8), df2['Monetary'].quantile(1)],
      labels = [5, 4, 3, 2, 1], include_lowest = True)

In [44]:
df2['Monetary'].quantile(0.75)

14481927.5

In [43]:
df2['Recency'].value_counts().sort_index()

0     37844
1     19894
2      6154
3     49885
4     80288
5     37126
6     41396
7     13836
8     11192
9      2250
10      113
11       22
Name: Recency, dtype: int64

In [146]:
df2['Monetary_5'].value_counts()

5    60000
4    60000
3    60000
2    60000
1    60000
Name: Monetary_5, dtype: int64

RFM (0,1) 2진 분류

In [155]:
threshold = df2['Recency'].quantile(0.8)
df2['Recency_2'] = df2['Recency'].apply(lambda x: 0 if x >= threshold else 1)

In [157]:
threshold = df2['Frequency'].quantile(0.8)
df2['Frequency_2'] = df2['Frequency'].apply(lambda x: 0 if x >= threshold else 1)

In [158]:
threshold = df2['Monetary'].quantile(0.8)
df2['Monetary_2'] = df2['Monetary'].apply(lambda x: 0 if x >= threshold else 1)

In [161]:
df2['Monetary_2'].value_counts()

1    240000
0     60000
Name: Monetary_2, dtype: int64

실제 하나카드 VVIP 기준 소비금액 상위 3000명

퍼스트 - 종합금액 7천 초과

로열 - 종합금액 3300만원 초과

클래식 - 종합금액 1700만원 초과

자체기준  0 -> 0.2,

VIP

충성고객

잠재 VIP 고객

잠재 충성 고객

놓치면 안될 고객

관심필요고객

이탈우려고객

겨울잠고객

VIP

충성고객

잠재 VIP 고객

잠재 충성 고객

놓치면 안될 고객

관심필요고객

이탈우려고객

겨울잠고객